In [2]:
from pathlib import Path
import numpy as np
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
# from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
import time
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from tqdm import tqdm
import pickle

In [2]:
def load_scatter_feature_files(feature_path):
    train_feats, train_labels, valid_feats, valid_labels, test_feats, test_labels = torch.load(feature_path)
    
    train_feats_flat = torch.flatten(train_feats, start_dim=1, end_dim=-1)
    train_feats_flat = np.array(train_feats_flat.cpu())
    train_labels = train_labels.cpu().data.numpy()
    valid_feats_flat = torch.flatten(valid_feats, start_dim=1, end_dim=-1)
    valid_feats_flat = np.array(valid_feats_flat.cpu())
    valid_labels = valid_labels.cpu().data.numpy()
    test_feats_flat = torch.flatten(test_feats, start_dim=1, end_dim=-1)
    test_feats_flat = np.array(test_feats_flat.cpu())
    test_labels = test_labels.cpu().data.numpy()
    

    return Bunch(train=(train_feats_flat, train_labels),
                 valid=(valid_feats_flat, valid_labels),
                 test=(test_feats_flat, test_labels))

In [23]:
# base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_{}.npy'
# base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_finetune_{}.npy'
# base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_finetune_2layers_{}.npy'

def load_resnet_feature_files(base_dir_resnet):
    test_feat_file = base_dir_resnet.format('test_feat')
    test_label_file = base_dir_resnet.format('test_label')

    train_feat_file = base_dir_resnet.format('train_feat')
    train_label_file= base_dir_resnet.format('train_label')

    valid_feat_file= base_dir_resnet.format('valid_feat')
    valid_label_file= base_dir_resnet.format('valid_label')

    test_feats = np.load(test_feat_file)
    test_labels = np.load(test_label_file)

    train_feats = np.load(train_feat_file)
    train_labels = np.load(train_label_file)

    valid_feats = np.load(valid_feat_file)
    valid_labels = np.load(valid_label_file)
    
    return Bunch(train=(train_feats, train_labels),
                 valid=(valid_feats, valid_labels),
                 test=(test_feats, test_labels))


In [3]:
from sklearn.neighbors import KNeighborsClassifier

def knn_classify(dataset):
    start = time.time()
    knn_classifier=KNeighborsClassifier()
    knn_classifier.fit(dataset.train[0], dataset.train[1])
    y_pred = knn_classifier.predict(dataset.test[0])
    end = time.time()
    print("knn", end - start)
    return y_pred

In [ ]:
def svm_classify(dataset):
#     n_estimators = 10
    start = time.time()
#     clf = OneVsRestClassifier(BaggingClassifier(svm.SVC(kernel='linear', probability=True), max_samples=1.0 / n_estimators, n_estimators=n_estimators), n_jobs=-1)
#     clf.fit(dataset.train[0], dataset.train[1])
#     y_pred = clf.predict(dataset.test[0])
#     print("Bagging SVC", end - start)
    svc = svm.SVC()
    svc.fit(dataset.train[0], dataset.train[1])
    y_pred = svc.predict(dataset.test[0])
    end = time.time()
    print("SVC", end - start)
    return y_pred

In [ ]:
fout = open('result/knn_resnet18_finetune_2layers','w')
base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_finetune_2layers_{}.npy'
dataset = load_resnet_feature_files(base_dir_resnet)
for k in [1,3,5,7]:
    y_pred = knn_classify(dataset,k)
    fout.write("Classification report for - \n{}:\n{}\n".format(
        str(k), metrics.classification_report(dataset.test[1], y_pred, digits=4)))

fout.close()   


fout = open('result/svm_resnet18_finetune_2layers','w')
base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_finetune_2layers_{}.npy'
dataset = load_resnet_feature_files(base_dir_resnet)
y_pred = svm_classify(dataset)
fout.write(metrics.classification_report(dataset.test[1], y_pred, digits=4))
fout.close()   

In [ ]:
fout = open('result/knn_resnet18_finetune','w')
base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_finetune_{}.npy'
dataset = load_resnet_feature_files(base_dir_resnet)
for k in [1,3,5,7]:
    y_pred = knn_classify(dataset,k)
    fout.write("Classification report for - \n{}:\n{}\n".format(
        str(k), metrics.classification_report(dataset.test[1], y_pred, digits=4)))

fout.close()   


fout = open('result/svm_resnet18_finetune','w')
base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_finetune_{}.npy'
dataset = load_resnet_feature_files(base_dir_resnet)
y_pred = svm_classify(dataset)
fout.write(metrics.classification_report(dataset.test[1], y_pred, digits=4))
fout.close()   

In [ ]:
fout = open('result/knn_resnet18','w')
base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_{}.npy'
dataset = load_resnet_feature_files(base_dir_resnet)
for k in [1,3,5,7]:
    y_pred = knn_classify(dataset,k)
    fout.write("Classification report for - \n{}:\n{}\n".format(
        str(k), metrics.classification_report(dataset.test[1], y_pred, digits=4)))

fout.close()   

fout = open('result/svm_resnet18','w')
base_dir_resnet = '/home/etsuko/MATH6380P/features/resnet18_{}.npy'
dataset = load_resnet_feature_files(base_dir_resnet)
y_pred = svm_classify(dataset)
fout.write(metrics.classification_report(dataset.test[1], y_pred, digits=4))
fout.close()  



In [ ]:

fout = open('result/svm_scatter','w')
for j in range(1,5):
    dataset = load_scatter_feature_files('features/scaterring_J'+str(j)+'.pt')
    y_pred = svm_classify(dataset)
    fout.write("Classification report for - \n{}:\n{}\n".format(
        str(j), metrics.classification_report(dataset.test[1], y_pred, digits=4)))

fout.close()   

fout = open('result/knn_scatter','w')
for j in range(1,5):
    fout.write(str(j)+'------------------------\n')
    dataset = load_scatter_feature_files('features/scaterring_J'+str(j)+'.pt')
    for k in [1,3,5,7]:
        y_pred = knn_classify(dataset,k)

        fout.write("Classification report for - \n{}:\n{}\n".format(
            str(k), metrics.classification_report(dataset.test[1], y_pred, digits=4)))

fout.close()  